# NonStaticCNN_긍부정 분류.ipynb
- `NonStaticCNN_모델 만들기.ipynb`에서 만든 비정태적 CNN 모델으로 리뷰 데이터를 긍부정 분류 한다. 
- 긍정 - 1, 부정 - 0

**필요한 파일** 
- `best_restaurant_semtiment_model.keras`
- `tokenizer.pkl`

In [ ]:
#'세션 다시 시작'이라는 안내메시지가 나오는 경우, "취소"하면 됨. (설치는 문제없이 되는듯??)
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install git+https://github.com/ssut/py-hanspell.git
!pip install konlpy
!pip install --upgrade tensorflow

In [2]:
import numpy as np
import pandas as pd
import io
import pickle
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Concatenate, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Kkma
from pykospacing import Spacing
from hanspell import spell_checker

In [6]:
# 4. 데이터 전처리

# 꼬꼬마 문장 분리 객체 생성
kkma = Kkma()

# 1. 전처리 함수: 특수문자 제거, 공백 처리, 맞춤법 교정
def preprocess_text(data):
    # 데이터프레임 형식이 아닌 경우 오류 처리
    if isinstance(data, (list, np.ndarray)):
        data = pd.DataFrame(data, columns=['reviews'])
    elif not isinstance(data, pd.DataFrame):
        raise ValueError("입력 데이터는 DataFrame, list, 또는 ndarray 형태여야 합니다.")

    # 한글, 공백, 언더스코어 외의 모든 문자 제거 (\n 포함)
    data['reviews'] = data['reviews'].str.replace(r"[^\sㄱ-ㅎㅏ-ㅣ가-힣]", " ", regex=True)

    # 전처리 후 남은 글자가 2글자 미만이라면 삭제
    data['reviews'] = data['reviews'].apply(lambda x: np.nan if isinstance(x, str) and len(x.strip()) <= 5 else x)
    
    #공백은 모두 nan으로 대체 
    data['reviews'].replace('', np.nan, inplace=True)
    
    # NaN 제거
    data.dropna(subset=['reviews'], inplace=True)

    return data

# 2. 문장 분리 함수: 꼬꼬마를 활용해 문장 단위로 분리
def split_sentences(data):
    if isinstance(data, list) or isinstance(data, np.ndarray):
        data = pd.DataFrame(data, columns=['reviews'])

    def safe_sentence_split(text):
        try:
            # 꼬꼬마로 문장 분리
            if isinstance(text, str):
                return kkma.sentences(text)
            return []
        except Exception as e:
            # 에러가 발생한 경우 원본을 로그로 남기고 리스트 형태로 반환
            print(f"문장 분리 중 오류 발생: {e}\n문제의 데이터: {text}")
            return [text]

    # 문장 분리 적용 (안전 처리)
    data['reviews'] = data['reviews'].apply(safe_sentence_split)
    data = data.explode('reviews', ignore_index=True)
    return data

# 3. 최종 데이터 전처리 함수
def preprocess_data(data, tokenizer=None, max_len=None):
    # 기존 전처리 적용
    data = preprocess_text(data)
    # 문장 분리 적용
    data = split_sentences(data)
    # 토큰화 및 패딩
    if tokenizer and max_len:

        data['reviews'] = data['reviews'].apply(lambda x: x if isinstance(x, str) else "")
        # 텍스트를 정수 시퀀스로 변환
        data['tokenized'] = tokenizer.texts_to_sequences(data['reviews'])
        # 패딩 추가
        data['tokenized'] = list(pad_sequences(data['tokenized'], maxlen=max_len, padding='post'))
    return data

Picked up JAVA_TOOL_OPTIONS: -Xmx4g -Xms512m


In [7]:
with open("/Users/minjiku/Desktop/비정태적CNN/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)
vocab_size = len(tokenizer.word_index) + 1
max_len=80

### 저장된 모델을 가져와서 라벨 찍기
- 아래 과정을 모든 리뷰 데이터에 대해 수행하면 된다. 

In [33]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. 저장된 모델 및 토크나이저 불러오기
model = load_model("/Users/minjiku/Desktop/비정태적CNN/best_restaurant_sentiment_model.keras")

# 2. 데이터 불러오기
unlabeled_data = pd.read_csv("/Users/minjiku/Desktop/비정태적CNN/maps.csv")
unlabeled_data = preprocess_data(unlabeled_data, tokenizer, max_len)
unlabeled_data.to_csv("/Users/minjiku/Desktop/비정태적CNN/maps_processed.csv", index=False)
# unlabeled_data = pd.read_csv("/content/drive/MyDrive/24-2Main/data/youtube_processed.csv", index_col=False)
X_unlabeled = unlabeled_data['reviews']

sequences = tokenizer.texts_to_sequences(X_unlabeled if isinstance(X_unlabeled, list) else X_unlabeled) #dataframe형태 보장
X_unlabeled = pad_sequences(sequences, maxlen=max_len)

# 4. 전처리
max_len = 80

# 5. 모델 예측
predictions = model.predict(X_unlabeled)

# 6. 예측 결과 저장
unlabeled_data['label'] = (predictions > 0.5).astype(int)  # 0: 부정, 1: 긍정

# 7. 결과 저장
unlabeled_data.to_csv("/Users/minjiku/Desktop/비정태적CNN/maps_labeled.csv", index=False)

print("예측 완료 및 저장 완료!")

/var/folders/v_/ccwz5s7931bf1s6yx_rzlzqm0000gn/T/ipykernel_51493/3337485792.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['reviews'].replace('', np.nan, inplace=True)


문장 분리 중 오류 발생: java.lang.OutOfMemoryError: Java heap space
문제의 데이터: 마즈크진바그어어어어어어어어어어어어어어어어어어어어어어어어어어
2487/2487 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step
예측 완료 및 저장 완료!


### 라벨을 바탕으로 식당별 긍부정 점수 계산
- 부정-0, 긍정-1 이므로 식당별 점수 평균을 계산하면, 긍정 점수의 비율이 된다. 

In [ ]:
import pandas as pd

#라벨이 포함된 데이터
file_path = ''
df = pd.read_csv(file_path)
    
#결과물 저장 경로
output_path = ''
df = df.groupby('restaurant')['label'].mean()
df = df.to_frame()
df = df.reset_index()
df.to_csv(output_path)